In [1]:
import pickle
import numpy as np
import pandas as pd


In [7]:
production_scale = pickle.load(open('production_scale.sav', 'rb'))
gross_scale = pickle.load(open('gross_scale.sav', 'rb'))
runtime_scale = pickle.load(open('runtime_scale.sav', 'rb'))
genres_pca = pickle.load(open('genres_pca.sav', 'rb'))


def full_genres( given  ):
    genres = np.array( ['action', 'adventure', 'aniimation', 'biography', 'comedy',
                       'crime', 'drama', 'family', 'fantasy', 'history', 'horror',
                       'music', 'musical', 'mystery', 'romance', 'sci-fi', 'sport',
                       'thriller', 'war', 'western'] )
    given = [cat.lower() for cat in given]
    genre_ip= np.zeros_like(genres, dtype=int)
    idx= [ np.where(genres == g )[0].tolist()[0] for g in given ] # returns the idx for the given genres
    genre_ip[idx]=1
    return genre_ip

def predict_gross( data ):
    model= pickle.load(open('model.sav', 'rb'))
    print('data is= \n', data)

    data['runtime']= runtime_scale.transform( np.array( data['runtime'] ).reshape(1, -1) )[0]
    print('data after runtime is= \n', data)

    data['production']= production_scale.transform( np.array( data['production']).reshape(1, -1) )[0]
    print('data after production is = \n', data)

    data['c1'], data['c2'], data['c3'] = zip( *genres_pca.transform( full_genres( data['genre'] ).reshape(1, -1).tolist() ) )
    print('data after pca is= \n', data)

    del data['genre']
    return gross_scale.inverse_transform( model.predict( pd.DataFrame(data)).reshape(1, -1)  ).tolist()[0][0]

In [8]:
data={}
data['runtime']= input('Enter runtime\n')
data['production']= input('Enter production price\n')
print('\nEnter genres carefully from here: \naction, adventure, aniimation, biography,\ncomedy, crime, drama, family, \nfantasy, history, horror, musis, \nmusical, mystery, romance, sci-fi, \nsport,thriller, war, western\n' )
data['genre']= [input('Enter genre\n') for i in range(3)]

print( f'\nPredicted gross= {np.floor(predict_gross(data))} $')



Enter genres carefully from here: 
action, adventure, aniimation, biography,
comedy, crime, drama, family, 
fantasy, history, horror, musis, 
musical, mystery, romance, sci-fi, 
sport,thriller, war, western

data is= 
 {'runtime': '122', 'production': '2165485', 'genre': ['war', 'action', 'crime']}
data after runtime is= 
 {'runtime': array([0.46280992]), 'production': '2165485', 'genre': ['war', 'action', 'crime']}
data after production is = 
 {'runtime': array([0.46280992]), 'production': array([-1.90430173]), 'genre': ['war', 'action', 'crime']}
data after pca is= 
 {'runtime': array([0.46280992]), 'production': array([-1.90430173]), 'genre': ['war', 'action', 'crime'], 'c1': (-217272.22678371976,), 'c2': (-30.109341040149616,), 'c3': (-115.0291093440213,)}

Predicted gross= 1162302.0 $
